#### S 0.0 Set parameter and import library

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow_addons as tfa
import time
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler, MinMaxScaler,Normalizer
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
# define parameters 
INPUT_SHAPE = (1000,90,1) 
ACTIVATION = None

In [ ]:
tf.keras.backend.clear_session()

# 1. Preparation

In [ ]:
### function define
lrelu = tf.keras.layers.LeakyReLU
softmax = tf.keras.layers.Softmax
normalization = tfa.layers.InstanceNormalization
identity = tf.keras.layers.Lambda(lambda x:x)

In [ ]:
# Gradient reverse layer (Yanin 2015)
@tf.custom_gradient
def grad_reverse(x):
    y = tf.identity(x)
    def custom_grad(dy):
        return -dy
    return y, custom_grad

class GradReverseLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(GradReverseLayer, self).__init__()

    def call(self, x):
        return grad_reverse(x)

In [ ]:
def build_encoder():
    inputs  = tf.keras.layers.InputLayer((1000,90,1))
    # Layer 1
    conv_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(5,5), strides = (5,5))
    norm_1 = tfa.layers.InstanceNormalization(axis=3,center=True,scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform")
    actv_1 = tf.keras.layers.ReLU()
    pool_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,1))
    # Layer 2
    conv_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides = (3,3))
    norm_2 = tfa.layers.InstanceNormalization(axis=3,center=True,scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform")
    actv_2 = tf.keras.layers.ReLU()
    pool_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,1))
    # Layer 3
    conv_3 = tf.keras.layers.Conv2D(filters=256, kernel_size=(2,2), strides = (2,2))
    norm_3 = tfa.layers.InstanceNormalization(axis=3,center=True,scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform")
    actv_3 = tf.keras.layers.Activation('tanh')
    pool_3 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,1))
    # Latent output
    latent = tf.keras.layers.Flatten()
    
    # model define
    mdl = tf.keras.models.Sequential([
        inputs,conv_1,norm_1,actv_1,pool_1,
        conv_2,norm_2,actv_2,pool_2,
        conv_3,norm_3,actv_3,pool_3,latent
    ])

    return mdl


In [ ]:
def build_classifier(latent_shape,output_shape):
    
    inputs  = tf.keras.layers.InputLayer(latent_shape)
    dense_1 = tf.keras.layers.Dense(128)
    actv_1  = tf.keras.layers.LeakyReLU()
    dense_2 = tf.keras.layers.Dense(output_shape)
    actv_2  = tf.keras.layers.Softmax()
    
    mdl = tf.keras.models.Sequential([inputs,dense_1,actv_1,dense_2,actv_2])
    return mdl
    
    

In [ ]:
def create_VGG(num=8,top=True):
    base_model = VGG16(include_top=False,input_shape=(1000, 90, 3))
    base_model.trainable = False
    inputs = tf.keras.layers.Input(shape=(1000, 90, 3))
    x = base_model(inputs, training=False)
    p = tf.keras.layers.GlobalAveragePooling2D()(x)
    if top == True:
        p = tf.keras.layers.Dense(128,activation='relu')(p)
        outputs = tf.keras.layers.Dense(num)(p)
        model = tf.keras.Model(inputs, outputs)
    else:
        model = tf.keras.Model(inputs, p)
    return model

In [ ]:
### Model 

encoder = build_encoder()
classifier = build_classifier(encoder.output_shape[1:],output_shape=8)
model = models.define_graph([encoder,classifier],INPUT_SHAPE)

### addition for DANN (gradient reverse)
# grad_reverse_layer = GradReverseLayer()
# dann_domain_discrimination_graph = model_utilis.define_graph([encoder,grad_reverse_layer,discriminator],INPUT_SHAPE)

### VGG 
# vgg_top = create_VGG(8,False)
# _,vgg_clf,vgg_discriminator = build_model(8,normalize=False,seperate=True,latent_shape=vgg_top.output_shape[1:])

# vgg_model = model_utilis.define_graph([vgg_top,vgg_clf],(1000,90,3))

print(model.summary())

In [ ]:
### Sample: 5,(1000,90,1) --> 5,(7,)

model(tf.random.uniform(shape=(5,1000,90,1)))

#### S1.3 Data

In [ ]:
import models,utils
from dataset import DatasetObject
from dataset import import_dataframe as import_experiment_data
from external.yousefi.dataset import import_dataframe as import_external_data
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# External dataset (Yousefi,2017)

folderpath = "./external/yousefi/Dataset/Data"
df = import_external_data(folderpath)

In [ ]:
# DatasetObject EXT

dataset_external = external_dataset.DatasetObject(df,cnn=True,sep='residual')

In [ ]:
type(OneHotEncoder())

In [ ]:
del df

In [ ]:
# Import Experiment 1
folderpath1 = "./data/exp_1"  # CHANGE THIS IF THE PATH CHANGED
df_exp1 = import_experiment_data(folderpath1) # CHANGE THIS IF THE PATH CHANGED

In [ ]:
# Preprocess for Experiment 1
df_exp1_X_ls,df_exp1_y_ls = [],[]
for user in df_exp1['user'].unique():
    dataframe = df_exp1[df_exp1['user']==user]
    features = dataframe[[f'amp_{i}' for i in range(1,91)]].to_numpy()
    features = MinMaxScaler().fit_transform(features)
    df_exp1_X_ls.append(features)
    label = dataframe[['label']].to_numpy()
    df_exp1_y_ls.append(label)

In [ ]:
# Dataset object 
dataset_exp1 = DatasetObject()
dataset_exp1.import_data(df_exp1_X_ls,df_exp1_y_ls,window_size=1000,slide_size=200,skip_labels=['noactivity'])

In [ ]:
# Reshape for CNN
dataset_exp1.data_transform(lambda arr: arr.reshape(*arr.shape,1),axis=1,col=0)
# Oversampling each fold 
# dataset_exp1.data_transform(lambda x,y,z : utils.resampling(x,y,z,True),axis=0,col=0)
# Encode label 
dataset_exp1.label_encode(1)
# Print shape
dataset_exp1.shape()

In [ ]:
del df_exp1

In [ ]:
# Self dataset EXP1 

folderpath1 = "./data/exp_1"  # CHANGE THIS IF THE PATH CHANGED
df_exp1 = dataset.import_dataframe(folderpath1) # CHANGE THIS IF THE PATH CHANGED
# processing step, required
df_exp1 = df_exp1.drop([f"theta_{i}" for i in range(1,91)], axis=1) 
print(df_exp1.shape)

dataset_exp1 = dataset.DatasetObject(df_exp1,cnn=True,stacking=False)
ohe_y = dataset_exp1.one_hot(1)
ohe_z = dataset_exp1.one_hot(2)

In [ ]:
# Self dataset EXP2 


folderpath2 = "./data/exp_2" # CHANGE THIS IF THE PATH CHANGED
df_exp2 = dataset.import_dataframe(folderpath2)
# Preprocessing 
df_exp2['user'] = df_exp2['user'].map(lambda x: x.split('.')[0]) # processing step, required
df_exp2 = df_exp2.iloc[:,2:] # processing step, required
print(df_exp2.shape)

dataset_exp2 = dataset.DatasetObject(df_exp2,cnn=True,stacking=False)
ohe_y = dataset_exp2.one_hot(1,ohe_y)
ohe_z = dataset_exp2.one_hot(2,ohe_z)

In [ ]:
df_ = pd.concat([df_exp1,df_exp2],axis=0)
dataset_exp4 = dataset.DatasetObject(df_,cnn=True,stacking=False)
ohe_y = dataset_exp4.one_hot(1)
ohe_z = dataset_exp4.one_hot(2)

In [ ]:
del df_exp1, df_exp2

# 2. Experiment

In [ ]:
tf.random.set_seed(1234)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=200)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
### single
train,test = dataset_exp1.query([6])
history = model.fit(x=train[0],y=train[1],batch_size=64,epochs=100,verbose=1,validation_data=(test[0],test[1]),callbacks=[callback])

### cross validation
# model,histories,scores = model_utilis.cross_validation(model,dataset_exp1,batch_size=64,epochs=10)

In [ ]:
# evaluation
train,test = dataset_exp1.query([6])
cmtx = utils.evaluation(model,test[0],test[1],ohe=dataset_exp1.encoders[0,1])
cmtx

# 3. Save

In [ ]:
current_time = datetime.now().strftime("%Y%m%d%H%M%S")
model_name = f"model_{current_time}"
savepath = f'./saved_model/{model_name}.h5'
model.save(savepath)
print("model saved: ",savepath)

In [ ]:
cmtx.to_csv(f"./record/cmtx_{model_name}.csv")